In [ ]:
# Install a conda package in the current Jupyter kernel
import sys
!conda install --yes -c conda-forge wordcloud 

In [1]:
import os
import re
import codecs
import pprint

In [2]:
pp = pprint.PrettyPrinter(indent=4)

In [3]:
home = os.path.expanduser("~")

In [4]:
bibFile = os.path.join(home,r"Downloads/bibitexAA.bib");

In [5]:
bibFile = os.path.join(home,r"Downloads/scopus.bib");

In [6]:
content ="";
with codecs.open(bibFile,'r') as f:
    content = f.read().decode("utf-8")

In [7]:
def parseBibtex(stringContent):
    entries = [];
    state = 0;
    key = "";
    value ="";
    entry = {};
    openBrackets = 0;

    for c in stringContent:
        if state == 0:
            #the beging of a new entry is detected so goes to next state
            if c == '@':
                state = 1;
                entry ={};
                continue;
            elif c in (',', ' ', '\n', '\t'):
                continue;

        elif state == 1:
            #Store entry type until a bracket could be found '{' then goes to next state
            if c == "{":
                entry['type'] = value;
                value ="";
                state = 2;
                continue;
        elif state == 2:
            #Read citation key until comma could be found
            if c == ",":
                entry['citationKey'] = value;
                value = "";
                state = 3;
                continue;
        elif state == 3:
            if c in ("", "="):
                #ignore
                continue;
            elif c == "{":
                #Read all caracteres until an open bracked could be found and store it as attribute key
                key = value.strip();
                value = "";
                state = 4;
                continue;
            elif c == "}":
                #Object end
                state =0;
                entries.append(entry);
                entry = {};
                continue;
            elif c in (',','\n'):
                continue;
        elif state == 4:
            #we are inside a attribute value which could have other '{'.
            #if an open bracket was '{' start counting it and if and close bracket appear decrease the counter.
            #the the counter is zero and an '{' was found than close the atribute value and return to previous state,
            #waiting
            if c == "{":
                openBrackets += 1;
            elif c =="}" and openBrackets !=0:
                openBrackets -= 1;
            elif c =="}" and openBrackets ==0:
                entry[key]=value.strip();
                key ="";
                value ="";
                state = 3;
                continue;
        value = value + c;
    return entries;

In [8]:
entries = parseBibtex(content);

# Análise da bibliografia

In [9]:
import pandas as pd;
import numpy as np;
import math;
from collections import defaultdict;
import matplotlib.pyplot as plt;
%matplotlib inline

In [10]:
df = pd.DataFrame(entries);
df['_citedby']=df.note.apply(lambda x: int(re.sub(r"cited By (\d+).*",r"\1",x)));
df['note']    =df.note.apply(lambda x: re.sub(r"(cited By \d+;?)(.*)",r"\2",x));
df['_journal']=df.journal.apply(lambda x: re.sub(r"\d+(st|th|rd)|20\d\d",r"",x));

In [11]:
#df.info()

In [12]:
df.drop(['abbrev_source_title','art_number','url',
         'page_count','coden','editor',
         'sponsors',
         'language',
         'isbn',
        'source'], axis=1, inplace=True)

In [15]:
df = df[[
    u'title',u'author',u'document_type', u'year', u'abstract', u'publisher', u'journal',
 u'author_keywords',
 'citationKey',
  u'affiliation',
 u'doi',
 u'issn',
 u'keywords',
 u'note',
 u'number',
 u'pages',
 u'references',
 'type',
 u'volume',
 '_citedby',
 '_journal']]

In [16]:
df.head()

,title,author,document_type,year,abstract,publisher,journal,author_keywords,citationKey,affiliation,...,issn,keywords,note,number,pages,references,type,volume,_citedby,_journal
0,On the role of words in the network structure ...,"Akimushkin, C. and Amancio, D.R. and Oliveira,...",Article,2018,Well-established automatic analyses of texts m...,Elsevier B.V.,Physica A: Statistical Mechanics and its Appli...,Authorship attribution; Burstiness; Complex ...,Akimushkin201849,"São Carlos Institute of Physics, University of...",...,03784371,"Complex networks; Linguistics; Semantics, Au...",,NaN,49-58,"Liang, W., Spectra of english evolving word co...",ARTICLE,495,0,Physica A: Statistical Mechanics and its Appli...
1,An ensemble scheme based on language function ...,"Onan, A.",Article,2018,Text genre classification is the process of id...,SAGE Publications Ltd,Journal of Information Science,ensemble learning; language function analysis...,Onan201828,"Department of Computer Engineering, Celal Baya...",...,01655515,Abstracting; Artificial intelligence; Comput...,,1,28-47,"Han, J., Kamber, M., (2006) Data mining: conce...",ARTICLE,44,0,Journal of Information Science
2,Document embeddings learned on various types o...,"Gómez-Adorno, H. and Posadas-Durán, J.-P. and ...",Article in Press,2018,"Recently, document embeddings methods have bee...",Springer-Verlag Wien,Computing,Authorship attribution; Doc2vec; Document em...,Gómez-Adorno20181,"Centro de Investigación en Computación (CIC), ...",...,0010485X,"Computer programming; Neural networks, Author...",Article in Press,NaN,1-16,NaN,ARTICLE,NaN,0,Computing
3,Modified principal component analysis for iden...,"Chu, K. and Liu, W. and She, Y. and Hua, Z. an...",Article,2018,Identification of the key environmental indica...,MDPI AG,Water (Switzerland),Coast; Key environmental indicators; Modifie...,Chu2018,Key Laboratory of Integrated Regulation and Re...,...,20734441,Biodiversity; Carbon; Coastal zones; Correl...,,1,NaN,"Yue, Q., Zhao, M., Yu, H.M., Xu, W., Ou, L., T...",ARTICLE,10,0,Water (Switzerland)
4,Analysis of multiple classifiers performance f...,"Baron, G.",Conference Paper,2018,In authorship attribution domain single classi...,Springer Science and Business Media Deutschlan...,"Smart Innovation, Systems and Technologies",Authorship attribution; Discretization; Ense...,Baron201833,"Silesian University of Technology, Akademicka ...",...,21903018,Authorship attribution; Discretizations; Ens...,Conference of 9th KES International Conferenc...,NaN,33-42,"Baron, G., Influence of data discretization on...",ARTICLE,73,1,"Smart Innovation, Systems and Technologies"


## Pesquisadores que participaram do maior número de publicações

In [18]:
authors = defaultdict(int);
for a in df.author.tolist():
    a = re.split(r"\s*and\s*",a);
    for aa in a:
        aa = aa.split(',');
        if len(aa) > 1:
            aa = aa[1]+" "+aa[0];
        else:
            aa = aa[0]
        authors[aa]+=1;

pd.DataFrame({
    "authors":authors.keys(), "count":authors.values()}).sort_values(by=['count',"authors"], ascending=False).head(20)

,authors,count
199,E. Stamatatos,11
547,M. Potthast,9
74,B. Stein,9
334,J. Savoy,8
621,G. Sidorov,7
289,M. Tschuggnall,5
34,W. Daelemans,4
417,U. Stańczyk,4
468,R. Kern,4
603,P. Rosso,4


In [20]:
authors = defaultdict(float);
maxYear = float(df.year.max())
for a,citation, year in zip(df.author.tolist(), df._citedby.tolist(), df.year.tolist()):
    a = re.split(r"\s*and\s*",a);
    factor = maxYear / int(year);
    for aa in a:
        aa = aa.split(',');
        if len(aa) > 1:
            aa = aa[1]+" "+aa[0];
        else:
            aa = aa[0]
        if aa in authors:
            authors[aa]+= citation * factor;
        else:
            authors[aa] = citation * factor;

pd.DataFrame({
    "authors":authors.keys(),
    "citation":authors.values()
}).groupby(by='authors').sum().reset_index().sort_values(by=['citation',"authors"], ascending=False).head(20)

,authors,citation
375,M. Potthast,78.077393
76,B. Stein,78.077393
149,E. Stamatatos,70.067967
383,M. Tschuggnall,37.025788
458,P. Rosso,34.030262
589,W. Daelemans,32.035226
172,F. Rangel,32.029270
565,T. Solorio,30.042182
268,J. Savoy,30.038706
369,M. Montes-Y-Gómez,29.041190


In [21]:
def cleanAuthor(x):
    authors ="";
    x = re.split(r"\s*and\s*",x);
    firstOccour = True;
    for author in x:
        author = author.split(',');
        if len(author) > 1:
            author = author[1]+" "+author[0];
        else:
            author = author[0];
        if not firstOccour:
            firstOccour = False;
            authors += " and "
        authors +=  author;
    return authors;
    
df.author = df.author.apply(lambda x: cleanAuthor(x));

In [22]:
df.head()

,title,author,document_type,year,abstract,publisher,journal,author_keywords,citationKey,affiliation,...,issn,keywords,note,number,pages,references,type,volume,_citedby,_journal
0,On the role of words in the network structure ...,C. Akimushkin D.R. Amancio O.N. Oliveira,Article,2018,Well-established automatic analyses of texts m...,Elsevier B.V.,Physica A: Statistical Mechanics and its Appli...,Authorship attribution; Burstiness; Complex ...,Akimushkin201849,"São Carlos Institute of Physics, University of...",...,03784371,"Complex networks; Linguistics; Semantics, Au...",,NaN,49-58,"Liang, W., Spectra of english evolving word co...",ARTICLE,495,0,Physica A: Statistical Mechanics and its Appli...
1,An ensemble scheme based on language function ...,A. Onan,Article,2018,Text genre classification is the process of id...,SAGE Publications Ltd,Journal of Information Science,ensemble learning; language function analysis...,Onan201828,"Department of Computer Engineering, Celal Baya...",...,01655515,Abstracting; Artificial intelligence; Comput...,,1,28-47,"Han, J., Kamber, M., (2006) Data mining: conce...",ARTICLE,44,0,Journal of Information Science
2,Document embeddings learned on various types o...,H. Gómez-Adorno J.-P. Posadas-Durán G. Sidoro...,Article in Press,2018,"Recently, document embeddings methods have bee...",Springer-Verlag Wien,Computing,Authorship attribution; Doc2vec; Document em...,Gómez-Adorno20181,"Centro de Investigación en Computación (CIC), ...",...,0010485X,"Computer programming; Neural networks, Author...",Article in Press,NaN,1-16,NaN,ARTICLE,NaN,0,Computing
3,Modified principal component analysis for iden...,K. Chu W. Liu Y. She Z. Hua M. Tan X. Liu L. ...,Article,2018,Identification of the key environmental indica...,MDPI AG,Water (Switzerland),Coast; Key environmental indicators; Modifie...,Chu2018,Key Laboratory of Integrated Regulation and Re...,...,20734441,Biodiversity; Carbon; Coastal zones; Correl...,,1,NaN,"Yue, Q., Zhao, M., Yu, H.M., Xu, W., Ou, L., T...",ARTICLE,10,0,Water (Switzerland)
4,Analysis of multiple classifiers performance f...,G. Baron,Conference Paper,2018,In authorship attribution domain single classi...,Springer Science and Business Media Deutschlan...,"Smart Innovation, Systems and Technologies",Authorship attribution; Discretization; Ense...,Baron201833,"Silesian University of Technology, Akademicka ...",...,21903018,Authorship attribution; Discretizations; Ens...,Conference of 9th KES International Conferenc...,NaN,33-42,"Baron, G., Influence of data discretization on...",ARTICLE,73,1,"Smart Innovation, Systems and Technologies"


In [23]:
df.groupby(by=["type", "document_type"])['type','abstract'].count()

type  abstract
type       document_type                   
ARTICLE    Article             92        89
           Article in Press     9         9
           Book Chapter         3         3
           Conference Paper    44        44
           Review               5         5
BOOK       Book Chapter         1         1
CONFERENCE Conference Paper   100       100

In [24]:
df.groupby(by=["type", "document_type"])['type','abstract'].count()

type  abstract
type       document_type                   
ARTICLE    Article             92        89
           Article in Press     9         9
           Book Chapter         3         3
           Conference Paper    44        44
           Review               5         5
BOOK       Book Chapter         1         1
CONFERENCE Conference Paper   100       100

In [26]:
df.groupby(by=["publisher"])['publisher','abstract'].count().sort_values(by=['abstract'], ascending=False).head(20)

,publisher,abstract
publisher,,
Springer Verlag,41,41
CEUR-WS,39,39
Institute of Electrical and Electronics Engineers Inc.,35,35
Oxford University Press,9,9
Elsevier B.V.,8,8
Association for Computational Linguistics (ACL),8,8
Elsevier Ltd,7,7
Springer New York LLC,6,6
Routledge,6,6


In [28]:
keywords = defaultdict(int);
for k in df.author_keywords.tolist():
    if  not isinstance(k,float):
        for kk in k.split(";"):
            kk = kk.upper();
            kk = re.sub(r"\b(\w+)(E?S)\b",r" \1",kk);
            kk = re.sub(r"\s{2,}"," ",kk).strip()
            keywords[kk]+=1;
    
pd.DataFrame({
    "keys": keywords.keys(),
    "counter":keywords.values()
}).groupby(by='keys').sum().reset_index().sort_values(by=['counter'], ascending=False).head(20)

,keys,counter
51,AUTHORSHIP ATTRIBUTION,50
40,AUTHOR IDENTIFICATION,23
444,STYLOMETRY,22
279,MACHINE LEARNING,18
468,TEXT MINING,10
58,AUTHORSHIP VERIFICATION,9
56,AUTHORSHIP IDENTIFICATION,9
464,TEXT CLASSIFICATION,8
138,DATA MINING,7
48,AUTHORSHIP,6


In [ ]:
keywords

In [29]:
df.to_csv(os.path.join(home,r"Downloads/scopus.csv"), index=False, encoding='utf-8')